## Quarterly Rebalance
### Fixed Allocation

#### IMPORTANT ACTIONS (Before Executing Program)

[MANUAL] Update the unit (shares of ETF currently held) in portfolio variable in CONSTANTS section

[OPTIONAL] Optionally update target weights for different fixed allocation

In [1]:
# Display pretty table for dataframe
from IPython.display import display

import pandas as pd
import numpy as np

import datetime

import yfinance as yf # https://github.com/ranaroussi/yfinance

In [2]:
# Initialize portfolio parameters

portfolio = {
    'TQQQ': {'targ_wt': 0.40, 'cur_unit': 119},
    'UPRO': {'targ_wt': 0.20, 'cur_unit': 18.04},
    #'MIDU': {'targ_wt': 0.10, 'cur_unit': 45},
    'TMF': {'targ_wt': 0.40, 'cur_unit': 709}
}

In [3]:
# Download last price from yFinance
tickers = list(portfolio.keys())
hist_df = yf.download(tickers, period = "5d", interval = "1d")

[*********************100%***********************]  3 of 3 completed


In [4]:
# convert portfolio to DF to keep track other datapoints
port_df = pd.DataFrame.from_dict(data = portfolio, orient='index')
port_df.index.name = 'ticker'
port_df['last_price'] = hist_df['Adj Close'].iloc[-1]
port_df['cur_val'] = port_df['cur_unit'] * port_df['last_price']
port_df['cur_wt'] = port_df['cur_val'] / port_df['cur_val'].sum()
port_df['last_date'] = hist_df.index[-1]

In [5]:
# Reblance portfolio
port_df['new_unit_reb'] = round(port_df['targ_wt'] * \
                                port_df['cur_val'].sum() / port_df['last_price'])

# Unit that need to be transacted to rebalance portfolio. (-)ve sell, (+)ve buy
# Requires no new funds to be added
port_df['trans_unit_reb'] = port_df['new_unit_reb'] - port_df['cur_unit']
display(port_df)

,targ_wt,cur_unit,last_price,cur_val,cur_wt,last_date,new_unit_reb,trans_unit_reb
ticker,,,,,,,,
TQQQ,0.4,119.00,30.24,3598.559973,0.281284,2022-08-26,169.0,50.00
UPRO,0.2,18.04,42.00,757.680000,0.059225,2022-08-26,61.0,42.96
TMF,0.4,709.00,11.90,8437.099730,0.659492,2022-08-26,430.0,-279.00


In [6]:
# Rebalance by adding/removing funds
port_df['fund_add'] = (port_df['cur_val'] / port_df['targ_wt']) - port_df['cur_val'].sum()

In [7]:
# Unit that need to be transacted, (-ve) sell, (+ve) buy
# The [trans_unit_+ticker] column shows the unit transacted when ticker units are not transacted
# The [fund_add] column value for row when ticker units are zero, shows fund that need to be 
# added/removed.

for ticker in port_df.index:
    new_unit = f"new_unit_{ticker}"
    trans_unit = f"trans_unit_{ticker}"
    port_df[new_unit] = \
    round((port_df.at[ticker, 'fund_add'] + \
           port_df['cur_val'].sum()) * port_df['targ_wt'] / port_df['last_price'])

for ticker in port_df.index:
    new_unit = f"new_unit_{ticker}"
    trans_unit = f"trans_unit_{ticker}"
    port_df[trans_unit] = port_df[new_unit] - port_df['cur_unit']
    
# Save to a CSV file for future reference
file_name = 'data/rebalance_{}.csv'.format(hist_df.index[-1].date())
port_df.to_csv(file_name)

display(port_df)

,targ_wt,cur_unit,last_price,cur_val,cur_wt,last_date,new_unit_reb,trans_unit_reb,fund_add,new_unit_TQQQ,new_unit_UPRO,new_unit_TMF,trans_unit_TQQQ,trans_unit_UPRO,trans_unit_TMF
ticker,,,,,,,,,,,,,,,
TQQQ,0.4,119.00,30.24,3598.559973,0.281284,2022-08-26,169.0,50.00,-3796.939770,119.0,50.0,279.0,0.00,-69.00,160.00
UPRO,0.2,18.04,42.00,757.680000,0.059225,2022-08-26,61.0,42.96,-9004.939702,43.0,18.0,100.0,24.96,-0.04,81.96
TMF,0.4,709.00,11.90,8437.099730,0.659492,2022-08-26,430.0,-279.00,8299.409622,302.0,127.0,709.0,-407.00,-582.00,0.00
